# GAN 示例
本示例由[AYLIEN/gan-intro](https://github.com/AYLIEN/gan-intro/blob/master/gan.py)简化而来。

首先进行库导入和一些设置。

In [1]:
import numpy as np
import tensorflow as tf

class parameters():
    def __init__(self, batch_size=8, hidden_size=4, num_steps=5000, log_every=100):
        self.batch_size = batch_size
        self.hidden_size = hidden_size
        self.num_steps = num_steps
        self.log_every = log_every

params = parameters()

seed = 13
np.random.seed(seed)
tf.set_random_seed(seed)

构建模型输入, 
- 待生成数据分布(正态分布)
- Generator输入（均匀分布白噪声）

In [4]:
def get_data(N=100, mu=4, sigma=0.5):
    return np.random.normal(mu, sigma, N)

def get_input(N=100, r=10):
    return np.linspace(-r, r, N) + np.random.random(N)*2/10/N

构建Generator、Discriminator:

In [ ]:
def linear(input, output_dim, scope=None, stddev=1.0):
    norm = tf.random_normal_initializer(stddev=stddev)
    const = tf.constant_initializer(0.0)
    with tf.variable_scope(scope or 'linear'):
        tf.get_variable('w', [input.get_shape()[1], output_dim], initializer=norm)
        tf.get_variable('b', [output_dim], initializer=const)
        return tf.matmul(input, w) + b

def generator(input, h_dim):
    h0 = tf.nn.softplus(linear(input, h_dim*2, 'g0'))
    h1 = linear(h0, 1, 'g1')
